In [4]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility/in_development/Will')
from cell_extractor.AnnotationProximityTool import DetectorMetricsDK55
from cell_extractor.CellDetector import MultiThresholdDetector

In [11]:
m55 = DetectorMetricsDK55()
detector = MultiThresholdDetector(animal = 'DK55',round =2)
detections = detector.load_detections()

In [10]:
det.head()

,x,y,section,name,mean,std,label
0,37633,18851,132,2200_sure,3.261358,1.963368,0.0
1,38135,17422,145,2200_sure,11.926437,1.838244,0.0
2,37838,17638,145,2200_sure,6.222859,1.584897,0.0
3,37725,17707,145,2300_sure,14.408781,1.403189,0.0
4,37634,17793,145,2300_sure,13.356399,1.703213,0.0


In [40]:
issure = [i.split('_')[1]=='sure' for i in detections.name]
isunsure = [i.split('_')[1]=='unsure' for i in detections.name]
sure = detections[issure]
unsure = detections[isunsure]
len(sure),len(unsure),len(detections)

(14010, 2024, 16034)

In [41]:
hqc = m55.load_human_qc()

In [42]:
import pandas as pd
sure = pd.DataFrame({'0':sure.x.astype('float'),'1':sure.y.astype('float'),'2':sure.section.astype('float'),'name':['sure' for _ in range(len(sure))]})
unsure = pd.DataFrame({'0':unsure.x.astype('float'),'1':unsure.y.astype('float'),'2':unsure.section.astype('float'),'name':['unsure' for _ in range(len(unsure))]})
sure.columns = hqc[0].columns
unsure.columns = hqc[0].columns

In [43]:
import numpy as np
all = pd.concat([sure,unsure]+hqc)
m55.annotations_to_compare =all
m55.annotations_to_compare.columns=['x','y','section','name']
m55.annotations_to_compare['x']=np.round(m55.annotations_to_compare['x'])
m55.annotations_to_compare['y']=np.round(m55.annotations_to_compare['y'])
m55.nannotations = len( m55.annotations_to_compare)
m55.find_equivalent_points() 

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 19203
after removing duplicates 16461


In [47]:
quantifications={}
category_name_and_criteria = {
    'Unverified detection'                  :lambda pair_categories : m55.check(pair_categories,include=['sure'],size_max=1),
    'Verified detection'                    :lambda pair_categories : m55.check(pair_categories,include=['sure','manual_train'],size_max=2,size_min=1),
    'unconfident identified  as positive'   :lambda pair_categories : m55.check(pair_categories,include=['unsure','manual_train'],size_max=2,size_min=1),
    'unconfident identified as negative'    :lambda pair_categories : m55.check(pair_categories,include=['unsure'],size_max=1),
    'a pair of detections at wrong locations'    :lambda pair_categories : m55.check(pair_categories,include=['sure','manual_train'],size_max=4,size_min=3)}
for category,criteria in category_name_and_criteria.items():
    quantifications[category] = m55.get_pairs_in_category(pair_category_criteria = criteria)

In [48]:
for i in quantifications:
    print(i)
    print(len(quantifications[i]))

Unverified detection
11838
Verified detection
1544
unconfident identified  as positive
36
unconfident identified as negative
1719
a pair of detections at wrong locations
100
